In [1]:
#import libraries
import numpy as np
import pandas as pd
import gzip
import requests
!pip install fastparquet
import fastparquet

     |████████████████████████████████| 1.5 MB 4.2 MB/s 
     |████████████████████████████████| 136 kB 21.5 MB/s 
     |████████████████████████████████| 1.6 MB 10.4 MB/s 


In [ ]:
#read feature space file to get target objects
features = pd.read_csv("metamdp_feature_space.csv")

targets_temp = np.unique(features['object'])
targets_temp1 = []

#removes digits from strings
for s in targets_temp:
  s_nodigits = []
  s_nodigits = [i for i in s if not i.isdigit()]
  targets_temp1.append(''.join(s_nodigits))

#print(targets_temp1)

targets = np.unique(targets_temp1)

#use all objects but 'Pad'
print(targets)

#take out the numbers!!! just get strings

['Airplane' 'Allergy Pills' 'Apple' 'Bacon' 'Banana' 'Beans' 'Block '
 'Book ' 'Bread' 'Candle' 'Candlestick' 'Carrot' 'Clipboard' 'Coconut'
 'Coffee' 'Crayon ' 'Cup' 'Cylinder ' 'Dodecahedron' 'Donut ' 'Eraser'
 'Gold' 'Helmet' 'Juice Box' 'Key' 'Knight' 'Laptop' 'Left Shoe'
 'Magazine ' 'Marker ' 'Milk Carton ' 'Mug' 'Notebook' 'Oil Bottle '
 'Oil Carafe ' 'Pad' 'Paint Can' 'Paper' 'Paper Bag' 'Paper Box' 'Pear'
 'Pen' 'Pencil ' 'Phone' 'Picture ' 'Pills' 'Pizza Box' 'Plastic Box '
 'Raw meat' 'Right Shoe' 'Rubber Duck' 'Scissors' 'Smartphone'
 'Soccer Ball' 'Soda Can' 'Takeout Box' 'Tea Box' 'Teabag' 'Tomato'
 'Towel' 'Train' 'Vase ' 'Water Bottle' 'Water Bottle ' 'Yogurt']


In [ ]:
landmarks = np.array(["counter","table","chair","floor"], dtype=object)

#total number of array of weights elements

total_weights = targets.shape[0]*landmarks.shape[0]

#weight matrix for each target,landmark combination intialized to zeroes
m = np.zeros((targets.shape[0],landmarks.shape[0]))

print(total_weights)
print(m)

260
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [2]:
#unzip conceptnet dataset (this takes a while!)
with gzip.open('conceptnet-assertions-5.7.0.csv.gz') as f:
    conceptnet = pd.read_csv(f,sep='\t',names=['uri','relation','start_node','end_node','info'])



FileNotFoundError: ignored

In [ ]:
#refer to concept net edge types to choose a relation: https://github.com/commonsense/conceptnet5/wiki/Relations
relation = '/r/AtLocation'

#choose the language of indexed concepts
language = '/c/en'

#keep only the data associated with chosen relation and language combo
conceptnet_new = conceptnet[conceptnet['relation'] == relation]
conceptnet_new = conceptnet_new[conceptnet_new['start_node'].str.contains(language) & conceptnet_new['end_node'].str.contains(language)]


In [ ]:
number_of_nonzero_weights = 0

for i in targets: 
  object =  targets[i]
  object = '/c/en/' + object

  #choose whether the object will be the start or end node relation
  node = 'start_node'

  for j in landmarks:
    subloc = '/c/en/' + landmarks[j]
    enode = 'end_node'
    #get relations for specified objects and associated info for each relationship (make more specific to object)
    object_relations = conceptnet_new[conceptnet_new[node] == object & conceptnet_new[enode] == subloc]
    #check if object_relations is empty or not, if so continue 
    if len(object_relations) == 0: 
      continue
    info = object_relations['info']
    for k in range(info.shape[0]):
      x = info.iloc[k].split(":")
      m[i][j] = x[len(x)-1][1:-1]
        
    if m[i][j] != 0:
      number_of_nonzero_weights+=1



print(number_of_nonzero_weights)


#check performance by counting number of zeros and dividing by total number of cells




NameError: ignored

In [ ]:
performance = number_of_nonzero_weights/total_weights